## Hybrid Search (Keyword + Vector ) & Reranking With Cohere API

In [ ]:
!pip install weaviate-client

In [2]:
!pip install langchain
!pip install langchain_community

In [3]:
import weaviate

/usr/local/lib/python3.10/dist-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [4]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key="xU1SAKsxas7UzYMZ77ctn60GI5sJf01cAcH8")

client = weaviate.Client(
  url="https://ydsq3xysmu8jbt4d4esta.c0.us-east1.gcp.weaviate.cloud",
  auth_client_secret=auth_config
)

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [11]:
client

In [12]:
client.is_ready()

True

In [13]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
# it is a class which combines vector and keyword search

In [9]:
client.schema.get()

{'classes': [{'class': 'RAG',
   'description': 'Documents for RAG',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the paragraph',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-huggingface': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'}],
   'replicationConfig': {'factor': 1},
   'shardingConfig': {'actualCount': 1,
    'actualVirtualCount': 128,
    'desiredCount': 1,
    'desiredVirtualCount': 128,
    'function': 'murmur3

In [10]:
client.schema.delete_all()

UnexpectedStatusCodeError: Delete class from schema! Unexpected status code: 403, with response body: {'error': [{'message': "forbidden: user 'general-wcs-readonly-api-key' has insufficient permissions to delete schema/objects"}]}.

In [ ]:
schema = {
    "classes": [
        {
            "class": "RAG",
            "description": "Documents for RAG",
            "vectorizer": "text2vec-huggingface",
            "moduleConfig": {"text2vec-huggingface": {"model": "sentence-transformers/all-MiniLM-L6-v2", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-huggingface": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

In [ ]:
client.schema.create(schema)

In [ ]:
client.schema.get()

In [ ]:
from langchain.vectorstores import Weaviate

retriever = Weaviate(
    client=client,

    index_name="hybrid",  # Replace with your Weaviate index name
    text_key="content"  # The key that represents the text field in your index
)

In [ ]:
model_name =  "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig , pipeline

In [ ]:
from langchain import HuggingFacePipeline

In [ ]:
 #function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

In [ ]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


In [ ]:
tokenizer = initialize_tokenizer(model_name)

In [ ]:
model = load_quantized_model(model_name)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    #max_length=2048,
    do_sample=True,
    top_k=5,
    max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [ ]:
doc_path ="/content/Retrieval-Augmented Generation for Large Language Models A Survey.pdf"

In [ ]:

llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
!pip install Pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader(doc_path)

In [ ]:
doc =loader.load()

In [ ]:
doc

In [ ]:
len(doc)

In [ ]:
!pip install langchain

In [ ]:
!pip install --upgrade langchain # upgrade langchain to the latest version
from langchain.text_splitter import RecursiveCharacterTextSplitter # import RecursiveCharacterTextSplitter


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size =100 , chunk_overlap=20)

In [ ]:
retriever.add_documents(doc)
